# Derivation of the DSRG-MRPT2 energy expression

The goal of this notebook is to use wicked to derive and implement the energy expression for the DSRG-MRPT2 method.

The energy can be expressed as
$$
E^\text{DSRG-MRPT2} = \langle \Phi|[\tilde{F}^{(1)} + \tilde{V}^{(1)}, \hat{T}_1 + \hat{T}_2 |\Phi\rangle
$$

In [1]:
import wicked as w
import numpy as np

import re

We begin by defining the orbital spaces for a CAS reference

In [2]:
w.reset_space()
w.add_space('c', 'fermion', 'occupied', ['m', 'n','o','p'])
w.add_space('a', 'fermion', 'general', ["u","v","w","x","y","z"])
w.add_space('v', 'fermion', 'unoccupied', ['e', 'f','g','h'])
wt = w.WickTheorem()

Next, we define a function to create a general operator

## Definition of the operators

In [3]:
Ftop = w.gen_op('Ft',1,'cav','cav',diagonal=False)
Vtop = w.gen_op('Vt',2,'cav','cav')
T1op = w.gen_op('T1',1,'av','ca',diagonal=False)
T2op = w.gen_op('T2',2,'av','ca',diagonal=False)

In [4]:
import re

def get_contraction_tensors(contraction):
    l = []
    m = re.search('einsum\(["\'][a-zA-Z,]*->[a-zA-Z,]*["\'],(.*)\)', contraction)
    if m:
        tensors = m.groups()[0].split(',')
        for t in tensors:
            m2 = re.search('([a-zA-Z0-9]+)\[["\'](\w*)["\']\]', t)
            if m2:
                tensor,indices = m2.groups()
                l.append((tensor,indices))
    return l

In [5]:
# this will store all the functions we will define
functions = []
tensors = []

def make_equation(op, functions, tensors):
    expr = wt.contract(w.rational(1,1),op, 0, 0)
    
    from IPython.display import display, Math
    display(Math(expr.latex()))

    mbeq = expr.to_manybody_equation("R")

    code = [f"def evaluate_energy_{len(functions)}():",
            "    # DSRG-MRPT2 energy term",
            "    R = 0.0"]
    for eq in mbeq[0]:
        contraction = eq.compile("einsum")
        if 'R' in contraction:
            code.append(f'    {contraction}')
            tensors.extend(get_contraction_tensors(contraction))
            
    code.append(f'    return R')
    funct = '\n'.join(code)

    print(f'Defined the function:\n\n{funct}\n')
    functions.append(funct)

    
F1T1 = w.commutator(Ftop,T1op)
make_equation(F1T1,functions,tensors)

F1T2 = w.commutator(Ftop,T2op)
make_equation(F1T2,functions,tensors)

V1T1 = w.commutator(Vtop,T1op)
make_equation(V1T1,functions,tensors)

V1T2 = w.commutator(Vtop,T2op)
make_equation(V1T2,functions,tensors)

for f in functions:
    exec(f)

<IPython.core.display.Math object>

Defined the function:

def evaluate_energy_0():
    # DSRG-MRPT2 energy term
    R = 0.0
    R += 1.000000000 * np.einsum("eu,ve,uv->",Ft["va"],T1["av"],gamma1["aa"])
    R += 1.000000000 * np.einsum("em,me->",Ft["vc"],T1["cv"])
    R += 1.000000000 * np.einsum("um,mv,vu->",Ft["ac"],T1["ca"],eta1["aa"])
    return R



<IPython.core.display.Math object>

Defined the function:

def evaluate_energy_1():
    # DSRG-MRPT2 energy term
    R = 0.0
    R += -0.500000000 * np.einsum("eu,wxve,uvwx->",Ft["va"],T2["aaav"],lambda2["aaaa"])
    R += -0.500000000 * np.einsum("um,mxvw,vwux->",Ft["ac"],T2["caaa"],lambda2["aaaa"])
    return R



<IPython.core.display.Math object>

Defined the function:

def evaluate_energy_2():
    # DSRG-MRPT2 energy term
    R = 0.0
    R += -0.500000000 * np.einsum("ue,xevw,vwux->",T1["av"],Vt["avaa"],lambda2["aaaa"])
    R += -0.500000000 * np.einsum("mu,wxmv,uvwx->",T1["ca"],Vt["aaca"],lambda2["aaaa"])
    return R



<IPython.core.display.Math object>

Defined the function:

def evaluate_energy_3():
    # DSRG-MRPT2 energy term
    R = 0.0
    R += 0.250000000 * np.einsum("uvef,efwx,wu,xv->",T2["aavv"],Vt["vvaa"],gamma1["aa"],gamma1["aa"])
    R += 0.125000000 * np.einsum("uvef,efwx,wxuv->",T2["aavv"],Vt["vvaa"],lambda2["aaaa"])
    R += 0.500000000 * np.einsum("muef,efmv,vu->",T2["cavv"],Vt["vvca"],gamma1["aa"])
    R += 0.250000000 * np.einsum("mnef,efmn->",T2["ccvv"],Vt["vvcc"])
    R += 0.500000000 * np.einsum("vwue,zexy,uz,xv,yw->",T2["aaav"],Vt["avaa"],eta1["aa"],gamma1["aa"],gamma1["aa"])
    R += 0.250000000 * np.einsum("vwue,zexy,uz,xyvw->",T2["aaav"],Vt["avaa"],eta1["aa"],lambda2["aaaa"])
    R += 1.000000000 * np.einsum("vwue,zexy,xv,uywz->",T2["aaav"],Vt["avaa"],gamma1["aa"],lambda2["aaaa"])
    R += -0.250000000 * np.einsum("vwue,zexy,uxyvwz->",T2["aaav"],Vt["avaa"],lambda3["aaaaaa"])
    R += 1.000000000 * np.einsum("mvue,xemw,ux,wv->",T2["caav"],Vt["avca"],eta1["aa"],gamma1["aa"])
    R += 1.000000000 * np.einsum("mvue

In [6]:
from collections import defaultdict
tensors_indices = defaultdict(set)
for tensor, label in tensors:
    tensors_indices[tensor].add(label)
    
L_blocks = ['va','ac','cv','vc']
Ft_blocks = tensors_indices['Ft']
Vt_blocks = tensors_indices['Vt']
T1_blocks = tensors_indices['T1']
T2_blocks = tensors_indices['T2'] | {'cava','aava','vaaa','aaca','vaca','ccav'}
H_blocks = Ft_blocks | Vt_blocks | T1_blocks | T2_blocks | {'cc', 'cccc', 'caca', 'aa', 'aaaa'}

```
    E0 (reference)                 =     -7.995916665442580
    <[F, T1]>                      =     -0.004054866166895
    <[F, T2]>                      =      0.005429854776796
    <[V, T1]>                      =      0.003047425118837
    <[V, T2]> (C_2)^4              =     -0.005258080443085
    <[V, T2]> C_4 (C_2)^2 HH       =      0.000003585009360
    <[V, T2]> C_4 (C_2)^2 PP       =      0.003072824360727
    <[V, T2]> C_4 (C_2)^2 PH       =     -0.003707054687659
    <[V, T2]> C_6 C_2              =      0.000426424806109
    <[V, T2]>                      =     -0.005462300954549
    DSRG-MRPT2 correlation energy  =     -0.001039887225811
    DSRG-MRPT2 total energy        =     -7.996956552668391
    max(T1)                        =      0.132328605821987
    max(T2)                        =      0.051519012097388
    ||T1||                         =      0.188661256796477
    ||T2||                         =      0.312662669906174
```

To begin our computation, we run a CASSCF computation on LiH using Forte's utility function `psi4_casscf`:

In [7]:
import forte
import forte.utils
from math import isclose

geom = """0 1
Li
H 1 1.6
symmetry c1
"""

basis = '6-31g'
mo_spaces = {'RESTRICTED_DOCC' : [1], 'ACTIVE' : [2]}

Ecasscf_psi4, wfn = forte.utils.psi4_casscf(geom, basis, mo_spaces)

print(f"CASSF energy (psi4):  {Ecasscf_psi4:.12f} Eh")

CASSF energy (psi4):  -7.995916665443 Eh


Next, use Forte's function `prepare_ints_rdms` to do the following:
- Prepare integrals
- Run a CASCI computation
- Compute the RDMs
- Canonicalize the MOs
- Rotate the integrals to the MO basis

In [8]:
Eref, mo_space_info, ints, rdms = forte.utils.prepare_ints_rdms(wfn, mo_spaces)
print(f"CASSF energy (forte): {Eref:.12f} Eh")
assert isclose(Ecasscf_psi4,Eref)

CASSF energy (forte): -7.995916665443 Eh


In [9]:
core = mo_space_info.corr_absolute_mo('RESTRICTED_DOCC')
actv = mo_space_info.corr_absolute_mo('ACTIVE')
virt = mo_space_info.corr_absolute_mo('RESTRICTED_UOCC')
hole = mo_space_info.corr_absolute_mo('GENERALIZED HOLE')
allm = mo_space_info.corr_absolute_mo('CORRELATED')

mos = {'c': core, 'a': actv, 'v': virt}
nmos = {k: len(v) for k, v in mos.items()}

nc, na, nv = nmos['c'], nmos['a'], nmos['v']

print(f"Core MOs: {mos['c']}; size: {nmos['c']}")
print(f"Active MOs: {mos['a']}; size: {nmos['a']}")
print(f"Virtual MOs: {mos['v']}; size: {nmos['v']}")

Core MOs: [0]; size: 1
Active MOs: [1, 2]; size: 2
Virtual MOs: [3, 4, 5, 6, 7, 8, 9, 10]; size: 8


Get the RDMs and the Hamiltonian

In [10]:
l1, l2, l3 = forte.spinorbital_cumulants(rdms)
lambda1 = {'aa' : l1}
lambda2 = {'aaaa' : l2}
lambda3 = {'aaaaaa' : l3}
gamma1 = {'aa' : l1}
eta1 = {'aa' : np.identity(gamma1['aa'].shape[0]) - gamma1['aa']}

def make_H(blocks):
    H = {}
    for block in blocks:
        if len(block) == 2:
            H[block] = forte.spinorbital_oei(ints,*[mos[b] for b in block])
        if len(block) == 4:
            H[block] = forte.spinorbital_tei(ints,*[mos[b] for b in block])            
    return H

H = make_H(H_blocks)

Next, we compute the reference energy to test the integrals and the RDMs

We compute the energy as
$$
E = V_\mathrm{NN} + \sum_m^\mathrm{C} h_{mm} + \frac{1}{2} \sum_{mn}^\mathrm{C} \langle mn \| mn \rangle +
 \frac{1}{2} \sum_{m}^\mathrm{C} \langle mu \| mv \rangle \lambda^{v}_{u}
+ \sum_{uv}^\mathrm{A} h^{u}_{v} \lambda^{v}_{u}
+ \frac{1}{4} \sum_{uvxy}^\mathrm{A} \langle uv \| xy \rangle \left( \lambda^{x}_{u} \lambda^{y}_{v} - \lambda^{x}_{v} \lambda^{y}_{u} + \lambda^{xy}_{uv} \right)
$$

In [11]:
Eref_test = ints.nuclear_repulsion_energy()
Eref_test += np.einsum('mm->',H['cc'])
Eref_test += 0.5 * np.einsum('mnmn->',H['cccc'])

Eref_test += np.einsum('mumv,vu->',H['caca'],lambda1['aa'])

Eref_test += np.einsum('uv,uv->',H['aa'],lambda1['aa'])
Eref_test += 0.25 * np.einsum('uvxy,xu,yv->',H['aaaa'],lambda1['aa'],lambda1['aa'])
Eref_test -= 0.25 * np.einsum('uvxy,xv,yu->',H['aaaa'],lambda1['aa'],lambda1['aa'])
Eref_test += 0.25 * np.einsum('uvxy,xyuv->',H['aaaa'],lambda2['aaaa'])

print(f"CASSF energy (forte): {Eref:.11f} Eh")
print(f"CASSF energy (forte): {Eref_test:.11f} Eh (spinorbital)")

CASSF energy (forte): -7.99591666544 Eh
CASSF energy (forte): -7.99591666544 Eh (spinorbital)


Compute the Fock matrix

In [12]:
def get_fock_block(p,q,rdms):
    f = forte.spinorbital_oei(ints,p,q)
    v = forte.spinorbital_tei(ints,p,core,q,core)
    f += np.einsum('piqi->pq', v)
    v = forte.spinorbital_tei(ints,p,actv,q,actv)
    f += np.einsum('piqj,ij->pq', v, gamma1['aa']) 
    return f

F = {'cc' : get_fock_block(core,core,rdms),
     'cv' : get_fock_block(core,virt,rdms),
     'ca' : get_fock_block(core,actv,rdms),
     'aa' : get_fock_block(actv,actv,rdms),
     'ac' : get_fock_block(actv,core,rdms),     
     'vc' : get_fock_block(virt,core,rdms),
     'va' : get_fock_block(virt,actv,rdms),
     'vv' : get_fock_block(virt,virt,rdms),
     'av' : get_fock_block(actv,virt,rdms),
    }

In [13]:
def compute_d1(F,f,spaces):
    fi = F[spaces[0] * 2].diagonal()
    fa = F[spaces[1] * 2].diagonal() 
    d = np.zeros((len(fi),len(fa)))
    for i in range(len(fi)):
        for a in range(len(fa)):
            if i % 2 == a % 2:
                d[i][a] = f(fi[i] - fa[a])  
    return d

def compute_d2(F,f,spaces):
    fi = F[spaces[0] * 2].diagonal()
    fj = F[spaces[1] * 2].diagonal()
    fa = F[spaces[2] * 2].diagonal()    
    fb = F[spaces[3] * 2].diagonal()
    d = np.zeros((len(fi),len(fj),len(fa),len(fb)))
    for i in range(len(fi)):
        for j in range(len(fj)):
            for a in range(len(fa)):
                for b in range(len(fb)):
                    if (i % 2) == (j % 2) == (a % 2) == (b % 2): # same spin case
                        d[i][j][a][b] = f(fi[i] + fj[j] - fa[a] - fb[b])
                    if ((i + j) % 2) * ((a + b) % 2) == 1: # opposite spin case
                        d[i][j][a][b] = f(fi[i] + fj[j] - fa[a] - fb[b])                        
    return d

In [14]:
s = 0.5

def denominator(x):
    return x

def regularized_denominator(x):
    if x == 0.0:
        return 0.0
    return (1. - np.exp(-s * x * x)) / x

Den = {'aa' : compute_d1(F,denominator,'aa')}
D1 = {b: compute_d1(F,regularized_denominator,b) for b in T1_blocks}
D2 = {b: compute_d2(F,regularized_denominator,b) for b in T2_blocks}

In [15]:
def one_plus_exp_s(x):
    return 1. + np.exp(-s * x * x)

K1 = {b: compute_d1(F,one_plus_exp_s,b) for b in Ft_blocks}
K2 = {b: compute_d2(F,one_plus_exp_s,b) for b in Vt_blocks}

In [16]:
T2 = {b: np.multiply(H[b],D2[b]) for b in T2_blocks}

T1 = {}
for block in T1_blocks:
    tmp = F[block].copy()
    tmp += np.einsum('xu,iuax,xu->ia',Den['aa'],T2[f'{block[0]}a{block[1]}a'],gamma1['aa'])
    T1[block] = np.multiply(tmp,D1[block]) 

In [17]:
def exp_s(x):
    return np.exp(-s * x * x)

L1 = {b: compute_d1(F,exp_s,b) for b in L_blocks}

def make_Ft(blocks):
    Ft = {}
    for block in blocks:
        Ft[block] = np.multiply(F[block],K1[block])
        tmp = np.einsum('xu,iuax,xu->ia',Den['aa'],T2[f'{block[0]}a{block[1]}a'],gamma1['aa'])
        Ft[block] += np.multiply(L1[block],tmp) 
    return Ft

Ft = make_Ft(Ft_blocks)

Vt = {b: np.multiply(H[b],K2[b]) for b in Vt_blocks}

In [18]:
Edsrg_corr = evaluate_energy_0() + evaluate_energy_1() + evaluate_energy_2() + evaluate_energy_3()

print(f'       DSRG-MRPT2 correlation energy  ={Edsrg_corr:20.12f}')
print(f'       DSRG-MRPT2 correlation energy  ={Edsrg_corr+0.001039887225811:20.12f} (error)')

       DSRG-MRPT2 correlation energy  =     -0.001039887226
       DSRG-MRPT2 correlation energy  =      0.000000000000 (error)


In [19]:
R = 0.0
R += 1.000000000 * np.einsum("eu,ve,uv->",Ft["va"],T1["av"],gamma1["aa"])
R += 1.000000000 * np.einsum("em,me->",Ft["vc"],T1["cv"])
R += 1.000000000 * np.einsum("um,mv,vu->",Ft["ac"],T1["ca"],eta1["aa"])
   
E_F_T1 = R    
print(f'       <[F, T1]>                      ={R:20.12f}')
print(f'       <[F, T1]> (error)              ={R+0.004054866166895:20.12f}')

       <[F, T1]>                      =     -0.004054866167
       <[F, T1]> (error)              =     -0.000000000000


In [20]:
R = 0.0
R += -0.500000000 * np.einsum("eu,wxve,uvwx->",Ft["va"],T2["aaav"],lambda2["aaaa"])
R += -0.500000000 * np.einsum("um,mxvw,vwux->",Ft["ac"],T2["caaa"],lambda2["aaaa"])

E_F_T2 = R
print(f'       <[F, T2]>                      ={R:20.12f}')
print(f'       <[F, T2]> (error)              ={R-0.005429854776796:20.12f}')

       <[F, T2]>                      =      0.005429854777
       <[F, T2]> (error)              =      0.000000000000


In [21]:
R = 0.0
R += 0.500000000 * np.einsum("weuv,xe,uvwx->",Vt["avaa"],T1["av"],lambda2["aaaa"])
R += 0.500000000 * np.einsum("vwmu,mx,uxvw->",Vt["aaca"],T1["ca"],lambda2["aaaa"])

E_V_T1 = R
print(f'       <[V, T1]>                      ={R:20.12f}')
print(f'       <[V, T1]> (error)              ={R-0.003047425118837:20.12f}')

       <[V, T1]>                      =      0.003047425119
       <[V, T1]> (error)              =      0.000000000000


In [22]:
R = 0.0
R += 0.250000000 * np.einsum("efuv,wxef,uw,vx->",Vt["vvaa"],T2["aavv"],gamma1["aa"],gamma1["aa"])
R += 0.500000000 * np.einsum("weuv,yzxe,xw,uy,vz->",Vt["avaa"],T2["aaav"],eta1["aa"],gamma1["aa"],gamma1["aa"])
R += 0.500000000 * np.einsum("efmu,mvef,uv->",Vt["vvca"],T2["cavv"],gamma1["aa"])
R += 1.000000000 * np.einsum("vemu,mxwe,wv,ux->",Vt["avca"],T2["caav"],eta1["aa"],gamma1["aa"])
R += 0.500000000 * np.einsum("vwmu,mzxy,xv,yw,uz->",Vt["aaca"],T2["caaa"],eta1["aa"],eta1["aa"],gamma1["aa"])
R += 0.250000000 * np.einsum("efmn,mnef->",Vt["vvcc"],T2["ccvv"])
R += 0.500000000 * np.einsum("uemn,mnve,vu->",Vt["avcc"],T2["ccav"],eta1["aa"])
R += 0.250000000 * np.einsum("uvmn,mnwx,wu,xv->",Vt["aacc"],T2["ccaa"],eta1["aa"],eta1["aa"])

E_V_T2_1 = R 
print(f'       <[V, T2]> (C_2)^4              ={R:20.12f}')
print(f'       <[V, T2]> (C_2)^4 (error)      ={R+0.005258080443085:20.12f}')

       <[V, T2]> (C_2)^4              =     -0.005258080443
       <[V, T2]> (C_2)^4 (error)      =      0.000000000000


In [23]:
R = 0.0

R += 0.125000000 * np.einsum("efuv,wxef,uvwx->",Vt["vvaa"],T2["aavv"],lambda2["aaaa"])
R += 0.250000000 * np.einsum("weuv,yzxe,xw,uvyz->",Vt["avaa"],T2["aaav"],eta1["aa"],lambda2["aaaa"])
R += 1.000000000 * np.einsum("weuv,yzxe,uy,vxwz->",Vt["avaa"],T2["aaav"],gamma1["aa"],lambda2["aaaa"])
R += 1.000000000 * np.einsum("vemu,mxwe,uwvx->",Vt["avca"],T2["caav"],lambda2["aaaa"])
R += 1.000000000 * np.einsum("vwmu,mzxy,xv,uywz->",Vt["aaca"],T2["caaa"],eta1["aa"],lambda2["aaaa"])
R += 0.250000000 * np.einsum("vwmu,mzxy,uz,xyvw->",Vt["aaca"],T2["caaa"],gamma1["aa"],lambda2["aaaa"])
R += 0.125000000 * np.einsum("uvmn,mnwx,wxuv->",Vt["aacc"],T2["ccaa"],lambda2["aaaa"])

E_V_T2_2 = R 
print(f'       <[V, T2]> C_4 (C_2)^2          ={R:20.12f}')
print(f'       <[V, T2]> C_4 (C_2)^2 (error)  ={R+0.000630645318:20.12f}')

       <[V, T2]> C_4 (C_2)^2          =     -0.000630645318
       <[V, T2]> C_4 (C_2)^2 (error)  =      0.000000000000


In [24]:
R = 0.0

R += -0.250000000 * np.einsum("weuv,yzxe,uvxwyz->",Vt["avaa"],T2["aaav"],lambda3["aaaaaa"])
R += 0.250000000 * np.einsum("vwmu,mzxy,uxyvwz->",Vt["aaca"],T2["caaa"],lambda3["aaaaaa"])

E_V_T2_3 = R 

print(f'       <[V, T2]> C_6 C_2              ={R:20.12f}')
print(f'       <[V, T2]> C_6 C_2 (error)      ={R-0.000426424806109:20.12f}')

       <[V, T2]> C_6 C_2              =      0.000426424806
       <[V, T2]> C_6 C_2 (error)      =     -0.000000000000


In [25]:
Edsrg = E_F_T1 + E_F_T2 + E_V_T1 + E_V_T2_1 + E_V_T2_2 + E_V_T2_3
print(f'       DSRG-MRPT2 correlation energy  ={Edsrg:20.12f}')
print(f'       DSRG-MRPT2 correlation energy  ={Edsrg+0.001039887225811:20.12f} (error)')
print(f'       DSRG-MRPT2 total energy        ={Eref+Edsrg:20.12f}')
assert isclose(Edsrg,-0.001039887225811)
assert isclose(Eref+Edsrg,-7.996956552668391)

       DSRG-MRPT2 correlation energy  =     -0.001039887226
       DSRG-MRPT2 correlation energy  =      0.000000000000 (error)
       DSRG-MRPT2 total energy        =     -7.996956552668
